In [1]:
import sys 
!{sys.executable} -m pip install tensorflow

In [6]:
import sys 
!{sys.executable} -m pip install opencv-python


In [18]:
import os
import numpy as np
import cv2
import keras.backend as K
import random
import tensorflow as tf


def load_image(Path = './valid/XR_ELBOW', size = 512):
    Images = []
    for path in Path:
        try:
            image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image,(size,size))
            Images.append(image)

        except Exception as e:
            print(str(e))

    Images = np.asarray(Images).astype('float32')
    Images=Images.reshape((len(Path),28,28,1))

    mean = np.mean(Images)			#normalization
    std = np.std(Images)
    Images = (Images - mean) / std
    return Images

In [19]:
def load_path(root_path = './valid/XR_ELBOW', size = 512):
	'''
	load MURA dataset
	'''

	Path = []
	labels = []
	for root,dirs,files in os.walk(root_path): #Read all pictures, os.walk Return to iterator genertor Traverse all files
		for name in files:
			path_1 = os.path.join(root,name)
			Path.append(path_1)
			if root.split('_')[-1]=='positive':	 #positive Label is 1，otherwise 0；
				labels+=[1]   	          	 #Last level directory file patient11880\\study1_negative\\image3.png
			else:
			    labels+=[0]
	print (len(Path))
	labels = np.asarray(labels)
	return Path, labels


Path,labels=load_path(r'F:\Bone Fracture Data\MURA-v1.1\train\XR_ELBOW',64)
images=load_image(Path,64)

4931


In [1]:
  def train_model():
        model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu,input_shape=(64,64,1)),
        tf.keras.layers.MaxPool2D((2,2),strides=1),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(128,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=2,padding='same'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=1),#no padding strides is one
        tf.keras.layers.Dropout(0.25),       
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=2,padding='same'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(6,activation=tf.nn.softmax)
        ])
        model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    
        model.fit(Images,labels,epochs=25,verbose=2)
     
        return model

In [51]:
model=trainModel()

Epoch 1/25
 - 5s - loss: 0.6671 - acc: 0.5916
Epoch 2/25
 - 5s - loss: 0.6480 - acc: 0.6151
Epoch 3/25
 - 5s - loss: 0.6386 - acc: 0.6364
Epoch 4/25
 - 5s - loss: 0.6245 - acc: 0.6567
Epoch 5/25
 - 5s - loss: 0.6096 - acc: 0.6627
Epoch 6/25
 - 5s - loss: 0.5912 - acc: 0.6820
Epoch 7/25
 - 5s - loss: 0.5672 - acc: 0.7076
Epoch 8/25
 - 5s - loss: 0.5401 - acc: 0.7274
Epoch 9/25
 - 5s - loss: 0.5154 - acc: 0.7443
Epoch 10/25
 - 6s - loss: 0.4854 - acc: 0.7587
Epoch 11/25
 - 5s - loss: 0.4565 - acc: 0.7840
Epoch 12/25
 - 5s - loss: 0.4190 - acc: 0.8102
Epoch 13/25
 - 5s - loss: 0.3963 - acc: 0.8197
Epoch 14/25
 - 5s - loss: 0.3555 - acc: 0.8471
Epoch 15/25
 - 5s - loss: 0.3312 - acc: 0.8580
Epoch 16/25
 - 5s - loss: 0.3289 - acc: 0.8578
Epoch 17/25
 - 5s - loss: 0.2829 - acc: 0.8808
Epoch 18/25
 - 5s - loss: 0.2572 - acc: 0.8927
Epoch 19/25
 - 5s - loss: 0.2381 - acc: 0.9012
Epoch 20/25
 - 5s - loss: 0.2216 - acc: 0.9144
Epoch 21/25
 - 5s - loss: 0.1938 - acc: 0.9242
Epoch 22/25
 - 6s - lo

In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               401536    
__________

In [53]:
#Validation of test data
ValidPath,Validlabels=load_path(r'F:\Bone Fracture Data\MURA-v1.1\valid\XR_ELBOW',64)
Validimages=load_image(ValidPath,64)

465


In [54]:
score = model.evaluate(Validimages, Validlabels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.3780374398795507
Test accuracy: 0.6408602


In [ ]:
model.save('elbow.h5')